In [29]:
import pandas as pd
import numpy as np
import datetime as dt

In [30]:
prices = pd.read_csv('C:/Users/psys0/Desktop/Projects/airbnb/airbnb_price.csv')
xls = pd.ExcelFile('C:/Users/psys0/Desktop/Projects/airbnb/airbnb_room_type.xlsx')
room_types = xls.parse()
reviews = pd.read_csv('C:/Users/psys0/Desktop/Projects/airbnb/airbnb_last_review.csv')


In [31]:
prices.head().style.background_gradient()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [32]:
room_types.head().style.background_gradient()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


In [33]:
reviews.head().style.background_gradient()

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


In [34]:
#cleaning

In [35]:
prices['price'] = prices['price'].str.replace(' dollars','')
prices['price'] = pd.to_numeric(prices['price'])
print(prices['price'].describe())

count    25209.000000
mean       141.777936
std        147.349137
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64


In [36]:
free_listing = prices['price'] == 0
prices = prices.loc[~free_listing]
avg_price = round(prices['price'].mean(),2)
print("The average price per night for an Airbnb listing in NYC is ${}.".format(avg_price))

The average price per night for an Airbnb listing in NYC is $141.82.


In [37]:
#adding new column for monthly cost calculation
# Calculate average_price_per_month
# Compare Airbnb and rental market

In [51]:
prices["price_per_month"] = prices["price"] * 365 / 12
average_price_per_month = round(prices["price_per_month"].mean(), 2)
print("Airbnb monthly costs are ${}, while in the private market you would pay {}.".format(average_price_per_month, "$3,100.00"))

Airbnb monthly costs are $4313.61, while in the private market you would pay $3,100.00.


In [40]:
room_types["room_type"] = room_types["room_type"].str.lower()
room_types["room_type"] = room_types["room_type"].astype("category")
room_frequencies = room_types["room_type"].value_counts()
print(room_frequencies)

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64


In [52]:
reviews["last_review"] = pd.to_datetime(reviews["last_review"])
first_reviewed = reviews["last_review"].dt.date.min()
last_reviewed = reviews["last_review"].dt.date.max()
print("The earliest Airbnb review is {}, the latest review is {}".format(first_reviewed, last_reviewed))


The earliest Airbnb review is 2019-01-01, the latest review is 2019-07-09


In [53]:
# Merge prices and room_types to create rooms_and_prices
# Merge rooms_and_prices with the reviews DataFrame to create airbnb_merged
# Drop missing values from airbnb_merged
# Check if there are any duplicate values

In [56]:
rooms_and_prices = prices.merge(room_types, how="outer", on="listing_id")
airbnb_merged = room_types.merge(rooms_and_prices, how="outer", on="listing_id")
airbnb_merged.dropna(inplace=True)
print("There are {} duplicates in the DataFrame.".format(airbnb_merged.duplicated().sum()))

There are 0 duplicates in the DataFrame.


In [57]:
airbnb_merged.head().style.background_gradient()

,listing_id,description_x,room_type_x,price,nbhood_full,price_per_month,description_y,room_type_y
0,2595,Skylit Midtown Castle,entire home/apt,225.000000,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,entire home/apt
1,3831,Cozy Entire Floor of Brownstone,entire home/apt,89.000000,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,200.000000,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,entire home/apt
3,5178,Large Furnished Room Near B'way,private room,79.000000,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,150.000000,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,entire home/apt


In [59]:
airbnb_merged["borough"] = airbnb_merged["nbhood_full"].str.partition(",")[0]
boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])
boroughs = boroughs.round(2).sort_values("mean", ascending=False)
print(boroughs)

                     sum    mean  median  count
borough                                        
Manhattan      1898963.0  184.03   149.0  10319
Brooklyn       1275435.0  122.02    95.0  10453
Queens          320765.0   92.81    70.0   3456
Staten Island    22974.0   86.04    71.0    267
Bronx            55231.0   79.24    65.0    697


In [60]:
# Create labels for the price range, label_names
label_names = ["Budget", "Average", "Expensive", "Extravagant"]
ranges = [0, 69, 175, 350, np.inf]
airbnb_merged["price_range"] = pd.cut(airbnb_merged["price"], bins=ranges, labels=label_names)
prices_by_borough = airbnb_merged.groupby(["borough", "price_range"])["price_range"].count()
print(prices_by_borough)

borough        price_range
Bronx          Budget          381
               Average         286
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5534
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5288
               Expensive      3073
               Extravagant     810
Queens         Budget         1632
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive        20
               Extravagant       0
Name: price_range, dtype: int64
